# Process Practice

**Uniprogramming**: only one process running at a time  
Two processes are not parallel and not concurrent, no matter what

**Multiprogramming**: allow multiple processes  
Two processes can run in parallel or concurrently

Modern OS try to run everything in parallel and concurrently

### The Scheduler Decides When to Switch

To create a process, the OS has to at least load it into memory

When it's waiting, the scheduler (coming later) decides when it’s running

### The Core Scheduling Loop Changes Running Processes

1. Pause the currently running process
2. Save its state, so you can restore it later
3. Get the next process to run from the scheduler
4. Load the next process’ state and let that run

There are 2 ways to do this:

- Let Processes Themselves
- The Operating System Pause

Cooperative multitasking  
The processes use a system call to tell the operating system to pause it

True multitasking  
The operating system retains control and pauses processes

Or, we can use interrupt

**Swapping process** --> Context Switching

- Save all the values, registers, or using stack (push, pop, etc.)
- Note: Context Switching purely means overhead (not our goal) --> Check if the program ever uses float, etc. for faster

## More IPC: A New API - `pipe`

    int pipe (int pipefd[2]);

Creates 2 new fd, 0 on success, -1 on failure 

Takes an int array of 2 int

--> Forms a one way comminication channel using 2 fds (that it newly opens)

--> Read end of the pipe, and write end of the pipe (transferring data)

#### Aside: Using & in Your Shell --> your shell will start that process and return imidiately

In [ ]:
%%file run.c

#include <errno.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

void check(int ret, const char* message) {
    if (ret != -1) {
        return;
    }
    int error = errno;      // More info: I want to exit and set my status to errno that causes the error
    perror(message);
    exit(error);
}

int main(void) {
    int fds[2];
    check(pipe(fds), "pipe");

    pid_t pid = fork();
    check(pid, "fork");
    if (pid > 0) {
        const char* str = "Howdy child";
        int len = strlen(str);
        int bytes_written = write(fds[1], str, len);
        check(bytes_written, "write");
    }
    else {
        char buffer[4096];
        int bytes_read = read(fds[0], buffer, sizeof(buffer));
        check(bytes_read, "read");
        printf("Child read: %.*s\n", bytes_read, buffer);
    }

    close(fds[0]);      
    close(fds[1]);

    return 0;
}

NOTE: `read` is blocking, meaning that even if `read` is run first from the child, and there's nothing to read yet, it will go to sleep. Then, after the parent `write`, the child process can still 

** pipe is not like a file, it will be stored in memory (buffer) by the kernel (not saved to a file)

*** // NOTE: after fork, both processes have fds opened --> Still have to close here for both ????? (are fds shared between processes or not?)

IF we do not write to parent: the parent will return, we will have an orphan that always keep waiting for the 'read' (NO TIMEOUT)  
NOTE: The child process still have not closed its write end of the pipe, so it doesn't get cleaned automatically by the kernel

We can modify the code to:

In [1]:
%%file run.c

#include <errno.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

void check(int ret, const char* message) {
    if (ret != -1) {
        return;
    }
    int error = errno;      // More info: I want to exit and set my status to errno that causes the error
    perror(message);
    exit(error);
}

int main(void) {
    int fds[2];
    check(pipe(fds), "pipe");

    pid_t pid = fork();
    check(pid, "fork");
    if (pid > 0) {
        const char* str = "Howdy child";
        int len = strlen(str);
        int bytes_written = write(fds[1], str, len);
        check(bytes_written, "write");
        close(fds[0]);      
        close(fds[1]);
    }
    else {      
        close(fds[1]);
        char buffer[4096];
        int bytes_read = read(fds[0], buffer, sizeof(buffer));
        check(bytes_read, "read");
        printf("Child read: %.*s\n", bytes_read, buffer);
        close(fds[0]);
    }

    return 0;
}

Overwriting run.c


This is because I closed fds[1] on the child side, so the `read` will not just stay there forever

"Think of fds like pointers, as 2 processes are forked, they have 2 fds that are representing the same 2 fds" BUT WE STILL HAVE TO CLOSE FROM BOTH PROCESSES